In [7]:
import sys

sys.path.append("../")

import MetaTrader5 as mt5
import pytz
import numpy as np
import pandas_ta as ta

from datetime import datetime
from terminal.Extensions.Mt5Helper import Mt5Helper

In [8]:
mt5.initialize()

True

In [9]:
slippage = 5
from_date = datetime(2024, 5, 23, tzinfo=pytz.utc)
to_date = datetime(
    from_date.year, from_date.month, from_date.day, 17, 59, 59, 0, tzinfo=pytz.utc
)
data = mt5.copy_ticks_range("WINM24", from_date, to_date, mt5.COPY_TICKS_TRADE)

In [11]:
rates = Mt5Helper.OHLC(data, "10s")
rates.index = rates.index.tz_localize(None)
rates.ta.stoch(append=True, k=28, d=12, smooth_k=7)

def plot_rates(length):
    rates.iloc[0:length][["open", "STOCHk_28_12_7", "STOCHd_28_12_7"]].plot(
        subplots=[("open",), ("STOCHk_28_12_7", "STOCHd_28_12_7")],
        figsize=(50, 20),
    )

plot_rates(length=500)
rates

,open,high,low,close,tick_volume,real_volume,STOCHk_28_12_7,STOCHd_28_12_7
time_msc,,,,,,,,
2024-05-23 09:00:50,126150.0,126225.0,126050.0,126160.0,15205,52334,NaN,NaN
2024-05-23 09:01:00,126160.0,126275.0,126150.0,126265.0,16691,52088,NaN,NaN
2024-05-23 09:01:10,126270.0,126295.0,126245.0,126270.0,8305,21995,NaN,NaN
2024-05-23 09:01:20,126265.0,126275.0,126165.0,126275.0,9316,28470,NaN,NaN
2024-05-23 09:01:30,126275.0,126340.0,126265.0,126330.0,7581,21067,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-05-23 17:59:10,125305.0,125310.0,125295.0,125295.0,164,362,37.245337,35.655843
2024-05-23 17:59:20,125300.0,125305.0,125290.0,125290.0,196,505,38.729931,35.832742
2024-05-23 17:59:30,125295.0,125330.0,125290.0,125330.0,1005,3101,46.756590,36.641327


In [12]:
sum_buy = 0
buy_qty = 0
buy_avg = 0
buy_profit = 0

sum_sell = 0
sell_qty = 0
sell_avg = 0
sell_profit = 0

net_qty = 0
net_profit = 0

used_slippage = 0
 
 
def update(i):
    rates.loc[i, "sum_buy"] = sum_buy
    rates.loc[i, "buy_qty"] = buy_qty
    rates.loc[i, "buy_avg"] = buy_avg
    rates.loc[i, "buy_profit"] = buy_profit
    rates.loc[i, "sum_sell"] = sum_sell
    rates.loc[i, "sell_qty"] = sell_qty
    rates.loc[i, "sell_avg"] = sell_avg
    rates.loc[i, "sell_profit"] = sell_profit
    rates.loc[i, "net_qty"] = net_qty
    rates.loc[i, "net_profit"] = net_profit
    rates.loc[i, "slippage"] = used_slippage


prev = None
for i, item in rates.iterrows():
    if np.isnan(item.open):
        update(i)
        continue

    if prev is None: 
        prev = item
        continue
    
    used_slippage = slippage

    if prev.STOCHk_28_12_7 < prev.STOCHd_28_12_7:
        qty = 1

        if net_qty < -2:
            qty = abs(net_qty) + 1

        sum_buy += qty * (item.open + used_slippage)
        buy_qty += qty

    if prev.STOCHk_28_12_7 > prev.STOCHd_28_12_7:
        qty = 1

        if net_qty > 2:
            qty = abs(net_qty) + 1

        sum_sell += qty * (item.open - used_slippage)
        sell_qty += qty

    buy_avg = 0 if sum_buy == 0 else sum_buy / buy_qty
    sell_avg = 0 if sum_sell == 0 else sum_sell / sell_qty

    buy_profit = 0 if buy_qty == 0 else (item.open - used_slippage - buy_avg) * buy_qty
    sell_profit = (
        0 if sell_qty == 0 else (sell_avg - item.open + used_slippage) * sell_qty
    )

    net_qty = buy_qty - sell_qty
    net_profit = buy_profit + sell_profit

    update(i)
    
    prev = item


rates.iloc[:][["net_profit"]].plot(figsize=(30, 10))

plot_rates(None)

In [6]:
rates.to_excel("output.xlsx")